# Toronto Clustering

The Wikipedia table is almost in the format we would like it to be in. Nonetheless, I treated it as if it was not and formatted it anyway, in order to demonstrate my solution for how it could be done.

First we have to import pandas and download the table from the Wikipedia entry.

In [1]:
import pandas as pd

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(url)[0]

We do not want any entries where "Borough" is "Not assigned".

In [2]:
df = df[df["Borough"]!="Not assigned"]

If "Neighbourhood" is "Not assigned", we want to replace it with the value in "Borough".

In [3]:
df["Neighbourhood"] = df.apply(lambda x: x["Borough"] 
                               if x["Neighbourhood"] == "Not assigned" 
                               else x["Neighbourhood"], axis=1)

We want to concatenate the "Neighbourhood" cells for which the "Postal Code"s are the same into a single row, where their strings get separated by comas. Finally, we show the first eleven rows of our dataframe.

In [4]:
replace = dict((key,"") for key in df[df.duplicated(subset="Postal Code",keep=False)]["Postal Code"].unique())

for index, row in df.iterrows():
    if row["Postal Code"] in replace.keys():
        if replace[row["Postal Code"]] == "":
            replace[row["Postal Code"]] += row["Neighbourhood"]
        else:
            replace[row["Postal Code"]] += (", "+row["Neighbourhood"])

df["Neighbourhood"] = df.apply(lambda x: replace[x["Postal Code"]]
                               if x["Postal Code"] in replace.keys() 
                               else x["Neighbourhood"], axis=1)

df.drop_duplicates(subset="Postal Code",keep="first",inplace=True)
df = df.reset_index(drop=True)
df.head(11)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


The last step is to print out the shape of our table.

In [5]:
df.shape

(103, 3)

Here I used the code snippet shown in the exercise, but changed the provider in order to get longitude and the latitude.

In [6]:
import geocoder # import geocoder

latitude = []
longitude = []

for i in range(0,df.shape[0]):
    # initialize your variable to None
    lat_lng_coords = None

    postal_code = df.iloc[i]["Postal Code"]

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.geocodefarm('Ontario {}, Toronto'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])


We then have to join the new information with the previous dataframe.

In [14]:
df_toronto = df.join(pd.DataFrame({"Latitude" : latitude, "Longitude" : longitude}))
df_toronto

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.756123,-79.329636
1,M4A,North York,Victoria Village,43.726780,-79.310738
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655354,-79.365044
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.721996,-79.445915
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.663910,-79.388733
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.652699,-79.511276
99,M4Y,Downtown Toronto,Church and Wellesley,43.666286,-79.382446
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.663506,-79.317429
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.633709,-79.496521


### Analysis Part

Here we start analysing the neighbourhood of Toronto and start by importing the libraries that we will need in the process.

In [10]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Here we provide our credentials to use the Foursquare API

In [11]:
CLIENT_ID = 'OFPFPKK1LXL1TT3WZLTFIWILFGMFDEE5T4B2BASI0BWFQW2P' # your Foursquare ID
CLIENT_SECRET = 'D4BYMB1FKXNDMH3D5P2O5BEP3IHPN3RBKLZ1FLXDZ3GNV5L5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
radius = 500 # The radius in which we will do the search

We now define a function toexplore the neighbourhood.

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

We use this function to create a new dataframe with the information about the nearby venues.

In [25]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

We now want to have a look at our data.

In [50]:
print(toronto_venues.shape)
toronto_venues.head(10)

(2391, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.756123,-79.329636,TTC Stop #09083,43.759655,-79.332223,Bus Stop
1,Parkwoods,43.756123,-79.329636,DVP at York Mills,43.758899,-79.334099,Intersection
2,Parkwoods,43.756123,-79.329636,Chick-N-Joy,43.759900,-79.326520,Fried Chicken Joint
3,Parkwoods,43.756123,-79.329636,TTC Stop 9083,43.759251,-79.334000,Bus Stop
4,Victoria Village,43.726780,-79.310738,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,Victoria Village,43.726780,-79.310738,Tim Hortons,43.725517,-79.313103,Coffee Shop
6,Victoria Village,43.726780,-79.310738,Pizza Nova,43.725824,-79.312860,Pizza Place
7,Victoria Village,43.726780,-79.310738,Eglinton Ave E & Sloane Ave/Bermondsey Rd,43.726086,-79.313620,Intersection
8,Victoria Village,43.726780,-79.310738,Wigmore Park,43.731023,-79.310771,Park
9,"Regent Park, Harbourfront",43.655354,-79.365044,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot


Counting the venues in each neighbourhood.

In [27]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,9,9,9,9,9,9
"Alderwood, Long Branch",2,2,2,2,2,2
"Bathurst Manor, Wilson Heights, Downsview North",4,4,4,4,4,4
Bayview Village,1,1,1,1,1,1
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
...,...,...,...,...,...,...
"Willowdale, Willowdale West",6,6,6,6,6,6
Woburn,4,4,4,4,4,4
Woodbine Heights,9,9,9,9,9,9


In [28]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 261 uniques categories.


We use one hot encoding in order to prepare the data four our clustering.

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Grouping the rows by neighbourhood and taking the mean of the occurrence frequency of each category

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.111111,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.041667,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,"Willowdale, Willowdale West",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
93,Woburn,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
94,Woodbine Heights,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
95,York Mills West,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


Printing each neighbourhood and theirtop 5 most common venues.

In [31]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0  Chinese Restaurant  0.22
1                Pool  0.11
2    Asian Restaurant  0.11
3        Noodle House  0.11
4                Bank  0.11


----Alderwood, Long Branch----
                   venue  freq
0  Performing Arts Venue   0.5
1              BBQ Joint   0.5
2      Accessories Store   0.0
3            Music Venue   0.0
4           Noodle House   0.0


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0  Middle Eastern Restaurant  0.25
1                       Park  0.25
2   Mediterranean Restaurant  0.25
3                Pizza Place  0.25
4                Music Store  0.00


----Bayview Village----
                             venue  freq
0                     Tennis Court   1.0
1                Accessories Store   0.0
2                           Office   0.0
3       Modern European Restaurant   0.0
4  Molecular Gastronomy Restaurant   0.0


----Bedford Park, Lawrence Manor East----
           

                 venue  freq
0          Coffee Shop  0.10
1     Sushi Restaurant  0.06
2          Yoga Studio  0.03
3  Distribution Center  0.03
4        Burrito Place  0.03


----Regent Park, Harbourfront----
                venue  freq
0         Coffee Shop  0.15
1  Italian Restaurant  0.08
2      Breakfast Spot  0.08
3         Yoga Studio  0.04
4         Event Space  0.04


----Richmond, Adelaide, King----
         venue  freq
0        Hotel  0.07
1  Coffee Shop  0.07
2         Café  0.06
3   Restaurant  0.05
4          Gym  0.04


----Rosedale----
                 venue  freq
0                 Park  0.14
1  Japanese Restaurant  0.14
2                 Bank  0.14
3          Coffee Shop  0.14
4       Sandwich Place  0.14


----Roselawn----
                             venue  freq
0                     Home Service   1.0
1                      Opera House   0.0
2       Modern European Restaurant   0.0
3  Molecular Gastronomy Restaurant   0.0
4              Monument / Landmark   0.0


-

We write a function to sort the venues in descending order.

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Createing the new dataframe and display the top 15 venues for each neighborhood.

In [53]:
num_top_venues = 15
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head(11)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Agincourt,Chinese Restaurant,Pool,Bank,Noodle House,Intersection,Supermarket,Shopping Mall,Asian Restaurant,Elementary School,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Donut Shop,Event Space,Falafel Restaurant
1,"Alderwood, Long Branch",BBQ Joint,Performing Arts Venue,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Donut Shop,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Middle Eastern Restaurant,Mediterranean Restaurant,Pizza Place,Park,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Fast Food Restaurant,Field,Filipino Restaurant
3,Bayview Village,Tennis Court,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Elementary School,Electronics Store,Eastern European Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Liquor Store,Restaurant,Thai Restaurant,Bakery,Bagel Shop,Pub,Sushi Restaurant,Juice Bar,Grocery Store,Hobby Shop,Video Store
5,Berczy Park,Coffee Shop,Seafood Restaurant,Italian Restaurant,Café,Japanese Restaurant,Beer Bar,Cocktail Bar,Hotel,Restaurant,Breakfast Spot,Bakery,Creperie,Park,Pizza Place,Art Gallery
6,"Birch Cliff, Cliffside West",Skating Rink,College Stadium,General Entertainment,Café,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant
7,"Brockton, Parkdale Village, Exhibition Place",Bar,Restaurant,Café,Coffee Shop,Gift Shop,Lounge,Arts & Crafts Store,Breakfast Spot,Bakery,Thrift / Vintage Store,Accessories Store,Cocktail Bar,Supermarket,French Restaurant,Boutique
8,"Business reply mail Processing Centre, South C...",Fast Food Restaurant,Harbor / Marina,Light Rail Station,Park,Restaurant,Farmers Market,Steakhouse,Liquor Store,Skate Park,Sushi Restaurant,Movie Theater,Sandwich Place,Italian Restaurant,Garden,Brewery
9,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Café,Italian Restaurant,Hotel,Restaurant,Park,Grocery Store,Spa,Gym,Seafood Restaurant,Cosmetics Shop,Sandwich Place,Pub,Spanish Restaurant,Peruvian Restaurant


Now we do a k-means clustering analysis. We want to identify 5 different clusters.

In [44]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 4, 1, 1, 1, 1, 1, 1], dtype=int32)

In [45]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood',how='right')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
78,M1S,Scarborough,Agincourt,43.792049,-79.270744,1,Chinese Restaurant,Pool,Bank,Noodle House,Intersection,Supermarket,Shopping Mall,Asian Restaurant,Elementary School,Eastern European Restaurant
93,M8W,Etobicoke,"Alderwood, Long Branch",43.599304,-79.536049,1,BBQ Joint,Performing Arts Venue,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Falafel Restaurant,Donut Shop
28,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.750977,-79.447235,1,Middle Eastern Restaurant,Mediterranean Restaurant,Pizza Place,Park,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Farmers Market
39,M2K,North York,Bayview Village,43.773895,-79.379799,4,Tennis Court,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant
55,M5M,North York,"Bedford Park, Lawrence Manor East",43.731823,-79.417870,1,Italian Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Liquor Store,Restaurant,Thai Restaurant,Bakery,Bagel Shop,Pub


In [46]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [47]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Finally, we want to display our results in a map.

In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters